# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [1]:

	import DataFrames as pd # pandas in Python
	import Distributions as dst
	import Plots as plts

In [2]:
begin
	include("./pmf.jl")
	import .ProbabilityMassFunction as pmf
end
